In [ ]:
import pandas as pd
from newspaper import Article
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

In [ ]:
article_links = []
def get_articles(soup):
    articles = soup.findAll('article',class_='LandingRiver_promoItem__LuiRv')
    for article in articles:
        article_link = article.find('a',class_='LandingRiver_titleLink__WHlTC')['href']
        article_links.append(article_link)

In [ ]:
URL = 'https://www.theatlantic.com/latest/'
next_url = URL
for i in range(250):
    response = requests.get(next_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml')
        get_articles(soup)
        a_tags=soup.findAll('a',class_='LandingRiver_paginationLink__QDaGM')
        if i==0:
            index=0
        else:
            index=1
        next_url = URL +  a_tags[index]['href'][8:]

In [ ]:
articles = {
    'Title': [],
    'Description': []
}
num_links = len(article_links)
part_size = num_links // 3
for link in tqdm(article_links[:part_size], total=part_size):
    news_article = Article(link)
    news_article.download()
    news_article.parse()
    title = news_article.title
    description = news_article.text
    articles['Title'].append(title)
    articles['Description'].append(description)

In [ ]:
for link in tqdm(article_links[part_size:2 * part_size], total=part_size):
    news_article = Article(link)
    news_article.download()
    news_article.parse()
    title = news_article.title
    description = news_article.text
    articles['Title'].append(title)
    articles['Description'].append(description)

In [ ]:
for link in tqdm(article_links[2 * part_size:], total=num_links - 2 * part_size):
    news_article = Article(link)
    news_article.download()
    news_article.parse()
    title = news_article.title
    description = news_article.text
    articles['Title'].append(title)
    articles['Description'].append(description)

In [ ]:
news_df = pd.DataFrame(articles)
news_df['Label'] = 1
news_df.to_csv('the_atlantic_news.csv',index=False)